In [1]:
import os
import sys

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import numpy as np

import clip
sys.path.append('../.')
from Metrics import nmi, acc, base_kmeans_model_evaluation, kmeans_with_init
from networks import CustomCLIP, load_clip_to_cpu
from lr_scheduler import ConstantWarmupScheduler
import faiss
import argparse
batch_size = 50
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from torch.autograd import Variable

In [2]:
backbone_name = "ViT-L/14"
device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
clip_model, preprocess = load_clip_to_cpu(backbone_name)
clip_model.to(device)
print('done')


done


In [3]:
trainset = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=preprocess) 

testset = torchvision.datasets.CIFAR10(root='../data', train=False,
                                        download=True, transform=preprocess)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=False)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False)
concatset = torch.utils.data.ConcatDataset([trainset, testset])
concatloader = torch.utils.data.DataLoader(concatset, batch_size=batch_size,
                                         shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
num_classes = len(testset.classes)

In [5]:
import time
def faiss_kmeans_model_evaluation(model, data_loader, num_cluster):
    model.eval()
    with torch.no_grad():
        start = time.time()
        dataset_size = len(data_loader.dataset)
        datas = np.zeros([dataset_size, 768])
        label_true = np.zeros(dataset_size)
        ii = 0
        for x, target in data_loader:
            b = x.shape[0]
            x = Variable(x).cuda()
            image_representation = model.encode_image(x)
            image_representation = image_representation / \
                image_representation.norm(dim=-1, keepdim=True)
            u = image_representation.cpu()
            datas[ii * data_loader.batch_size:(ii + 1) *
                  data_loader.batch_size, :] = u.data.numpy()
            label_true[ii * data_loader.batch_size:(ii + 1)
                       * data_loader.batch_size] = target.numpy()
            ii = ii + 1
        print(time.time()-start)
#         kmeans = KMeans(n_clusters=num_cluster, random_state=0).fit(datas)
        return datas, label_true

In [ ]:
with torch.no_grad():
    datas,label_true = faiss_kmeans_model_evaluation(
        clip_model, trainloader, num_classes)
# np.save('../npy_folder/test_cluster.npy', test_label)

In [109]:
kmeans = faiss.Kmeans(d=datas.shape[-1], k=num_classes, niter=10, verbose=True,gpu=False)

In [110]:
datas = datas.astype(np.float32)
kmeans.train(datas)

Sampling a subset of 2560 / 50000 for training
Clustering 2560 points in 768D to 10 clusters, redo 1 times, 10 iterations
  Preprocessing in 0.02 s


447.2464599609375

In [111]:
# datas = datas.astype(np.float32)
dists, label_pred = kmeans.index.search(datas, 1)

In [112]:
print(label_pred.shape)

(50000, 1)


In [113]:
ACC = acc(label_true, label_pred, 10)
NMI = nmi(label_true, label_pred.squeeze(axis=-1))

In [114]:
print(ACC,NMI)

0.82898 0.8682115603685252


In [106]:
# kmeans = KMeans(n_clusters=10, random_state=0).fit(datas)
# label_pred = kmeans.labels_
# centroids = kmeans.cluster_centers_
ACC = acc(label_true, label_pred, 10)
NMI = nmi(label_true, label_pred)
print('image_ACC', ACC)
print('image_NMI', NMI)

image_ACC 0.95328
image_NMI 0.9012859197933579


In [1]:
import pandas as pd
pd.DataFrame([1])

,0
0,1
